<a href="https://colab.research.google.com/github/fm-add9/practice/blob/main/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%B7%D0%B0%D1%89%D0%B8%D1%82%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Статистический анализ


In [1]:
import numpy as np
import pandas as pd

data = {
    'feature1': [10, 12, 14, 1000, 15, 13, 11],  # Аномалия: 1000
    'feature2': [1, 2, 3, 4, 5, 6, 7]
}
df = pd.DataFrame(data)

print(df.describe())

# Поиск аномалий с помощью межквартильного размаха (IQR)
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Определяем аномалии
anomalies = ((df < (Q1 - 1.5 * IQR)) | ((df > (Q3 + 1.5 * IQR))))
print("Аномалии:")
print(anomalies)

          feature1  feature2
count     7.000000  7.000000
mean    153.571429  4.000000
std     373.243824  2.160247
min      10.000000  1.000000
25%      11.500000  2.500000
50%      13.000000  4.000000
75%      14.500000  5.500000
max    1000.000000  7.000000
Аномалии:
   feature1  feature2
0     False     False
1     False     False
2     False     False
3      True     False
4     False     False
5     False     False
6     False     False


2. Методы машинного обучения для обнаружения аномалий

Алгоритмы машинного обучения: Isolation Forest, One-Class SVM или DBSCAN.

In [2]:
from sklearn.ensemble import IsolationForest

# Модель Isolation Forest
model = IsolationForest(contamination=0.1, random_state=42)  # contamination — доля аномалий
model.fit(df)

# Предсказание аномалий
df['anomaly'] = model.predict(df)
print("Результат обнаружения аномалий:")
print(df)

# Аномалии будут помечены как -1
anomalies = df[df['anomaly'] == -1]
print("Обнаруженные аномалии:")
print(anomalies)

Результат обнаружения аномалий:
   feature1  feature2  anomaly
0        10         1        1
1        12         2        1
2        14         3        1
3      1000         4       -1
4        15         5        1
5        13         6        1
6        11         7        1
Обнаруженные аномалии:
   feature1  feature2  anomaly
3      1000         4       -1


4. DBSCAN

Алгоритм кластеризации, который может использоваться для обнаружения выбросов.

In [3]:
from sklearn.cluster import DBSCAN

# Применяем DBSCAN
dbscan = DBSCAN(eps=3, min_samples=2)  # eps — максимальное расстояние между точками
df['anomaly'] = dbscan.fit_predict(df[['feature1', 'feature2']])

# Аномалии будут помечены как -1
anomalies = df[df['anomaly'] == -1]
print("Обнаруженные аномалии:")
print(anomalies)

Обнаруженные аномалии:
   feature1  feature2  anomaly
3      1000         4       -1



5. Автоматическое обнаружение аномалий с помощью библиотеки PyOD


In [4]:
!pip install pyod
from pyod.models.knn import KNN  # Используем метод k-ближайших соседей

model = KNN()
model.fit(df[['feature1', 'feature2']])

# 1 — нормальные данные, 0 — аномалии
df['anomaly'] = model.predict(df[['feature1', 'feature2']])

# Аномалии будут помечены как 0
anomalies = df[df['anomaly'] == 0]
print("Обнаруженные аномалии:")
print(anomalies)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-2.0.3-py3-none-any.whl size=200466 sha256=a0a83cbd3a8400ddd09d9d5063c7ee7ddd2aed91f1922634f8d3d3e34187a78e
  Stored in directory: /root/.cache/pip/wheels/2d/60/5b/f74eccd2c9c892a2c298202ca510f10995f9940647fcc2d97f
Successfully built pyod
Обнаруженные аномалии:
   feature1  feature2  anomaly
0        10         1        0
1        12         2        0
2        14         3        0
4        15         5        0
5        13         6        0
6        11         7        0
